In [ ]:
# %% [markdown]
# # CIRA Simple Workflow Example
# 
# This notebook demonstrates a simple workflow using the CIRA library, covering:
# 1. Setting up API keys.
# 2. Fetching historical stock data.
# 3. Running a basic strategy backtest.
# 
# ## 1. API Key Setup
# 
# To use CIRA with Alpaca Markets for live trading or to fetch most data, you need to set your API keys.
# You can do this in two ways:
# 
# **Method 1: Set keys directly in your script (DO NOT COMMIT YOUR KEYS TO VERSION CONTROL)**

# %% 
import cira
import os

# Option 1: Set keys directly (replace with your actual keys)
# Ensure you have a .env file or environment variables set for APCA_ID and APCA_KEY if you run this often
# or store them securely.
# cira.auth.set_api_keys("YOUR_APCA_ID", "YOUR_APCA_KEY")

# Option 2: Use a key file (JSON format)
# Create a JSON file (e.g., /path/to/your/alpaca_keys.json) with:
# {
#   "APCA-API-KEY-ID": "YOUR_APCA_ID",
#   "APCA-API-SECRET-KEY": "YOUR_APCA_KEY"
# }
# cira.auth.set_key_file_path("/path/to/your/alpaca_keys.json")

# Option 3: Use environment variables (Recommended for security)
# CIRA will automatically try to load APCA_ID and APCA_KEY from your environment variables.
# This is the most secure way if you are running this in a shared environment or version control.
# For this example, we'll try to use them if they exist, otherwise, it will rely on manual setup above.

apca_id = os.getenv("APCA_ID")
apca_key = os.getenv("APCA_KEY")

if apca_id and apca_key:
    print("Using API keys from environment variables.")
    cira.auth.set_api_keys(apca_id, apca_key)
else:
    print("API keys from environment variables not found.")
    print("Please uncomment one of the methods above (set_api_keys or set_key_file_path) and provide your keys.")
    print("Alternatively, set APCA_ID and APCA_KEY environment variables.")
    # As a fallback for this example to run without live keys for some parts (like crypto data):
    # We will proceed, but live trading and most stock data will fail.
    # cira.auth.set_api_keys("YOUR_APCA_ID_FALLBACK", "YOUR_APCA_KEY_FALLBACK") # Replace or ensure this fails gracefully

# Check if keys are set and appear valid
# Note: check_keys() makes an actual API call to verify.
# For some data (like crypto), keys might not be strictly required by Alpaca.
# However, for trading and most stock data, they are essential.
print("\nChecking API key status...")
try:
    keys_are_valid = cira.auth.check_keys()
    if keys_are_valid:
        print("API keys are set and appear to be valid.")
    else:
        print("API keys are set but seem invalid, or the check failed.")
        print("If you haven't set your keys, this is expected.")
        print("You might still be able to fetch some data (e.g., crypto) depending on Alpaca's current policies.")
except ValueError as e:
    print(f"Error during key check: {e}")
    print("This usually means API keys are not set at all.")
    print("Please ensure your API keys are configured using one of the methods above.")
    print("You might still be able to fetch some data (e.g., crypto) depending on Alpaca's current policies.")

# For demonstration, we'll print the configuration CIRA will attempt to use
# (This does not print the actual secret key for security)
try:
    id_to_use, _ = cira.auth.get_api_keys() # This will raise ValueError if no keys are found by any method
    if id_to_use:
        print(f"CIRA is configured to use API Key ID starting with: {id_to_use[:4]}...")
    else:
        # This case should not be reached if get_api_keys raises ValueError as expected
        print("CIRA is not configured with any API keys (get_api_keys returned empty but did not error).")
except ValueError:
    # This is the expected path if no keys are configured by any method.
    print("CIRA is not configured with any API keys (get_api_keys raised ValueError).")

In [ ]:
# %% [markdown]
# ## 2. Fetching Historical Stock Data
# 
# Once your API keys are set up (or if you are fetching data that doesn't require keys, like some crypto data), you can fetch historical market data.
# 
# **Note:** Fetching historical stock data generally requires valid API keys. If keys are not set up correctly, the following code may fail.

# %%
import cira
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

# Ensure matplotlib plots are displayed inline in the notebook
# %matplotlib inline # This is a magic command for IPython environments

# %%
# Define the stock symbol and timeframe
STOCK_SYMBOL = "AAPL"  # Apple Inc. as an example
END_DATE = datetime.now().date()
START_DATE = END_DATE - timedelta(days=365) # Data for the last year

print(f"Fetching historical data for {STOCK_SYMBOL} from {START_DATE} to {END_DATE}\n")

# Create a Stock object
try:
    stock_asset = cira.Stock(STOCK_SYMBOL)

    # Fetch historical data
    # The library might handle fetching data differently based on whether keys are present
    # For stocks, keys are generally needed.
    historical_data_df = stock_asset.historical_data_df(start_date=START_DATE, end_date=END_DATE)

    if historical_data_df.empty:
        print(f"No data returned for {STOCK_SYMBOL}. This could be due to incorrect API key setup, market holidays, or issues with the data provider.")
    else:
        print(f"Successfully fetched {len(historical_data_df)} data points for {STOCK_SYMBOL}.")
        print("\nDataFrame Head:")
        print(historical_data_df.head())
        print("\nDataFrame Tail:")
        print(historical_data_df.tail())

        # Plotting the closing prices
        if 'close' in historical_data_df.columns:
            plt.figure(figsize=(12, 6))
            historical_data_df['close'].plot(title=f'{STOCK_SYMBOL} Closing Prices ({START_DATE} to {END_DATE})')
            plt.xlabel("Date")
            plt.ylabel("Price (USD)")
            plt.grid(True)
            plt.show()
        else:
            print("\n'close' column not found in historical data. Cannot plot closing prices.")
            print("Available columns:", historical_data_df.columns)

except ValueError as e:
    print(f"Error creating Stock object or fetching data: {e}")
    print("This often occurs if API keys are not properly configured or if the symbol is invalid.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
    print("This could be due to various reasons including network issues or problems with the Alpaca API service.")

# %% [markdown]
# **Note on Data Access:**
# * If you encounter errors above, double-check your API key setup from Step 1.
# * Alpaca provides free data for IEX, but for full data access (including NBBO), you might need a paid data subscription with Alpaca.
# * For crypto data (e.g., `cira.Cryptocurrency("BTC/USD")`), API keys might not always be required by Alpaca for historical data.

In [ ]:
# %% [markdown]
# ## 3. Basic Strategy Backtest
# 
# Now, let's use the historical data we fetched to run a simple backtest with one of CIRA's built-in strategies.
# We'll use the `ByAndHold` strategy as an example and compare it against itself (though `back_test_against_buy_and_hold` is designed for comparing a custom strategy to BuyAndHold).
# 
# **Note:** The backtesting module requires a DataFrame with specific column names if you are using features. For price data, it typically expects columns like 'open', 'high', 'low', 'close', 'volume'. The `historical_data_df` from Alpaca usually provides these. For `back_test`, the `asset_prices` parameter should be a DataFrame where each column represents an asset and contains its price (e.g., closing price).

# %%
import cira
import cira.strategy as cstrat # Using an alias for cira.strategy
import pandas as pd
import matplotlib.pyplot as plt

# Ensure matplotlib plots are displayed inline
# %matplotlib inline

# %%
# We assume 'historical_data_df' exists from the previous cell.
# Let's check if it exists and is not empty.
if 'historical_data_df' in locals() and not historical_data_df.empty:
    print("Using historical data fetched in the previous step for backtesting.")

    # Prepare data for backtesting:
    # The back_test function expects:
    # 1. feature_data: A DataFrame that can be used by the strategy's iterate method.
    #    For simple strategies like BuyAndHold, this might just be the price data itself.
    # 2. asset_prices: A DataFrame where each column is an asset and values are prices (e.g., 'close' or 'open').
    
    # For this example, we'll use the 'close' prices for our `asset_prices`
    # and the full OHLCV data as `feature_data`.
    if 'close' not in historical_data_df.columns:
        print("Error: 'close' column not found in historical_data_df. Cannot proceed with backtest.")
        print("Available columns:", historical_data_df.columns)
    else:
        asset_prices_df = historical_data_df[['close']].copy() 
        # Rename column to the asset symbol for clarity if backtesting multiple assets later,
        # though for a single asset backtest with default strategy, it might not be strictly necessary.
        # asset_prices_df.rename(columns={'close': STOCK_SYMBOL}, inplace=True)
        
        feature_data_df = historical_data_df.copy()

        # Instantiate a strategy
        # Simple BuyAndHold strategy
        buy_and_hold_strategy = cstrat.ByAndHold()
        
        # Dollar Cost Averaging strategy (invest 100 units of cash at each step, if possible)
        # dca_strategy = cstrat.DollarCostAveraging(amount=100) 

        print(f"\nRunning backtest for {buy_and_hold_strategy.name} strategy...")
        
        # Initial capital for the backtest
        INITIAL_CAPITAL = 100000.0

        try:
            # Run the backtest
            # We use back_test here. back_test_against_buy_and_hold is useful
            # if you have a custom strategy and want a quick comparison.
            portfolio_history_df = cstrat.back_test(
                strat=buy_and_hold_strategy,
                feature_data=feature_data_df, 
                asset_prices=asset_prices_df, # DataFrame of close prices
                capital=INITIAL_CAPITAL,
                use_fees=True # Set to False if you don't want to simulate Alpaca's fee rate
            )

            print("\nBacktest completed.")
            print("Portfolio history (tail):")
            print(portfolio_history_df.tail())

            # Plot the portfolio value over time
            plt.figure(figsize=(12, 6))
            portfolio_history_df[buy_and_hold_strategy.name].plot(title=f'{buy_and_hold_strategy.name} Strategy Performance')
            plt.xlabel("Date")
            plt.ylabel("Portfolio Value (USD)")
            plt.grid(True)
            plt.show()
            
            # You can also use multi_strategy_backtest to compare multiple strategies
            # For example:
            # strats_to_compare = [buy_and_hold_strategy, dca_strategy]
            # multi_results_df = cstrat.multi_strategy_backtest(strats_to_compare, feature_data_df, asset_prices_df, INITIAL_CAPITAL)
            # multi_results_df.plot(title=\"Strategy Comparison\", figsize=(12,6)).grid(True)
            # plt.show()

        except Exception as e:
            print(f"An error occurred during the backtest: {e}")
            print("This could be due to issues with the data format or the strategy implementation.")

else:
    print("Historical data (historical_data_df) not available or empty. Skipping backtest section.")
    print("Please ensure the previous cell (Fetching Historical Data) ran successfully.")

# %% [markdown]
# ### Disclaimer for Backtesting
# The results of any backtest are based on historical data and **do not guarantee future performance**.
# Financial markets are inherently uncertain, and various factors can influence actual trading results.
# This backtest is provided for educational and informational purposes only. Users should exercise caution 
# and conduct additional research before applying any trading strategy in live markets.
# CIRA's backtesting also includes a disclaimer within its `back_test` function documentation.

In [ ]:
# %% [markdown]
# ## 4. Conclusion and Next Steps
# 
# This notebook has demonstrated a basic workflow with CIRA:
# 1.  **API Key Setup:** Crucial for most functionalities. Remember to handle your keys securely.
# 2.  **Fetching Data:** Showed how to get historical stock data. CIRA also supports fetching data for options and cryptocurrencies.
# 3.  **Backtesting:** Ran a simple `ByAndHold` strategy. You can create your own custom strategies by inheriting from `cira.strategy.Strategy` and implementing the `iterate` method. Explore other built-in strategies and the `multi_strategy_backtest` function.
# 
# ### Further Exploration:
# *   **Different Assets:** Try fetching data for other stocks, options, or cryptocurrencies (e.g., `cira.Cryptocurrency("BTC/USD")`).
# *   **Custom Strategies:** Develop and test your own trading strategies. Refer to the CIRA documentation and examples for guidance.
# *   **Live Trading:** Once you are comfortable with your strategies (after thorough backtesting and paper trading), you might explore CIRA's capabilities for placing trades through Alpaca. **Always exercise extreme caution with live trading and understand the risks involved.**
# *   **Error Handling:** This notebook includes basic error handling. Robust applications would require more comprehensive error management.
# *   **Advanced Features:** Explore other CIRA modules for portfolio management, exchange information, etc.
# 
# We hope this example helps you get started with CIRA! Check out the official [CIRA GitHub repository](https://github.com/AxelGard/cira) for more examples, documentation, and discussions.

# %%
# This is the end of the example notebook.
print("Simple CIRA workflow example complete.")